First we will be setting up all imports and set up basic logging:

In [ ]:
from memflow import *
import logging

FORMAT = "%(levelname)s %(name)s %(asctime)-15s %(filename)s:%(lineno)d %(message)s"
logging.basicConfig(format=FORMAT)
logging.getLogger().setLevel(logging.INFO)

The first step towards memflow is then creating an Inventory. The Inventory contains a list of all connectors found on the System:

In [ ]:
inventory = Inventory()

The next step is creating an actual connector. Memflow connectors are used to acquire physical memory of a device. Examples of connectors are: kvm, qemu, pcileech, coredump, ...

The arguments are provided as triplet, they use the following form:

```
{connector}:{target}:{arg1},{arg2},{arg3}
```

Where `target` is the name of the target (in case the connector supports multiple targets).
Where `arg1`, `arg2` and `arg3` use a `key=value` format like `device=FPGA`

Here we are loading the `kvm` connector and letting it connect to a virtual machine with the name `win11` as the target.

In [ ]:
connector = inventory.create_connector(name="kvm", args=":win11:")

Without the `target` argument the kvm connector will just pick the first virtual machine it finds. It is also possible on some connectors to retrieve a list of all available targets (whereas the resulting `name` is the name of the target):

In [ ]:
inventory.connector_target_list("qemu")

It is also possible to retrieve a Help-Text for Plugins, this is especially useful when writing CLI applications:

In [19]:
inventory.connector_help("qemu")

'The `qemu` connector implements a memflow plugin interface\nfor QEMU on top of the Process Filesystem on Linux.\n\nThis connector requires access to the qemu process via the linux procfs.\nThis means any process which loads this connector requires\nto have at least ptrace permissions set.\n\nThe `target` argument specifies the target qemu virtual machine.\nThe qemu virtual machine name can be specified when starting qemu with the -name flag.\n\nAvailable arguments are:\nmap_base: override of VM memory base\nmap_size: override of VM memory size'

In [20]:
inventory.os_help("win32")

ERROR memflow.error 2022-12-18 21:00:09,824 error.rs:31 connector: not supported (Os-Plugin `win32` does not support help text.)


Exception: connector: not supported

The previously created connector can now be utilized to initialize an Os. In the given example we try to find Windows running in memory in the KVM Virtual Machine.

In [ ]:
os = inventory.create_os(name="win32", input=connector)

You can now access drivers and processes on the target. In the windows (and memflow) world drivers are just modules of the root process (in the case of Windows all drivers are modules of the ntoskrnl.exe kernel process). The following example shows how to list all running drivers:

In [ ]:
drivers = os.module_info_list()
print(drivers)

In [ ]:
kernel = os # TODO:

To access the memory of a driver or process you have to open the corresponding process:

In [ ]:
process = os.process_from_name("explorer.exe")

A Process also features the same functions for retrieving modules:

In [ ]:
modules = process.module_info_list()
print(modules)

It is also possible to get a module by it's name:

In [ ]:
module = process.module_from_name("Explorer.EXE")

Finally we are able to read Data from the process/module. In the following example we read parts of the COFF Header from the PE Header of the primary module:

In [ ]:
# Define COFF Header struct and read it from the module:
class COFFHeader(Structure):
    _fields_ = [
        ("_pad0x0", c_uint8 * 6),
        ("sections", c_uint16),
        ("timestamp", c_uint32),
    ]

header = process.read(module.base + 0x40, COFFHeader)
print(header)